## P2 Data Engineer Code Challenge

### 0. Setup

Here we will import packages, set the working directory and specify any custom functions to assist with the coding operations.

In [9]:
# Import libraries
import os
import urllib # Modules for working with URLs
import json # Modules that handles JSON data interchange format
import pandas as pd # Modules used to enhance data manipulation
import numpy as np # Modules for advanced mathematics

# Specify for exporting results
output_path = os.getcwd()

# Simplify the data pulling
def create_df(limit = "1000", contain = ""):
    # Setup the basis of the query
    url = "https://data.gov.au/data/api/3/action/datastore_search?"
    resource_id = "resource_id=809c77d8-fd68-4a2c-806f-c63d64e69842"
    query = url + resource_id
    if len(limit) != 0:
        # Add limit if specified
        query = query + "&limit=" + limit
    if len(contain) != 0:
        # Add restraints if specified
        query = query + "&q=" + contain
    
    # Pull data and deserialise
    fileobj = urllib.request.urlopen(query)
    data = json.loads(fileobj.read())
    # Generate dataframe
    df = pd.DataFrame.from_dict(data['result']['records'])
    
    return df

### 1. How many passengers flew into and out of Australia for each airline in each month of 2019?


In [10]:
# Create dataframe
pass_2019 = create_df(limit = "10000", contain = "2019")

# Subset to required variables
pass_2019 = pass_2019[['Airline', 'Month_num', 
                       'Passengers_In', 'Passengers_Out']]
# Convert 'Month_num' to int to correct ordering on display
pass_2019 = pass_2019.astype({'Month_num': np.int})
pass_2019 = pass_2019.sort_values('Month_num')

Display a subset of the passengers coming in and out of Australia in 2019 based on airline and ordered by month.

In [11]:
# Outputs subset of results 
display(pass_2019.head()) 
display(pass_2019.tail())

,Airline,Month_num,Passengers_In,Passengers_Out
0,Solomon Airlines,1,1690,1411
109,Federal Express Corporation,1,0,0
116,Qantas Airways,1,2394,2019
117,Qantas Airways,1,72093,65699
646,SriLankan Airlines,1,8550,7431


,Airline,Month_num,Passengers_In,Passengers_Out
568,Korean Air,12,16894,18520
983,AirAsia X,12,46737,67575
544,Virgin Australia,12,507,580
526,China Airlines,12,5009,6269
1078,Emirates,12,0,0


The results look correct. Export the reuslts to a csv in the working directory.

In [12]:
pass_2019.to_csv(output_path + "\\passengers_in_out_aus_2019.csv")

### 2. In the last 6 months which port country was responsible for the most passengers coming into the country?

In [13]:
# Create dataframe
# Luckily for us, the last 6 months are the first 6 months of 2021, so we only
# need to filter by year being 2021
pass_2021 = create_df(limit = "10000", contain = "2021")

# Let's store the summary of passengers in by port country in a new dataframe
pass_2021_sum = pd.DataFrame(columns = ['Port_Country', 'Total_Passengers_In'])

# By port country, calculate total passengers coming in
for port in pass_2021['Port_Country'].unique():
    pass_nums = pass_2021.Passengers_In[pass_2021.Port_Country == port]
    pass_sum = pd.to_numeric(pass_nums).sum()
    pass_2021_sum = pass_2021_sum.append(
        pd.DataFrame([[port, pass_sum]], 
                     columns = ['Port_Country', 'Total_Passengers_In']), 
        ignore_index=False)
    
# Display port with most passengers in
max_pass = pass_2021_sum['Total_Passengers_In'].max()
display(pass_2021_sum.loc[pass_2021_sum['Total_Passengers_In'] == max_pass])

,Port_Country,Total_Passengers_In
0,New Zealand,46413


### 3. What is the 3 month rolling average of freight arriving to Australia for each airline throughout 2018?

In [14]:
# Create dataframe
freight_2018 = create_df(limit = "10000", contain = "2018")
freight_2018 = freight_2018[['Airline', 'Freight_In_(tonnes)', 'Month_num']]
freight_2018.columns = ['Airline', 'Freight_In', 'Month']
# Convert 'Month_num' to int and 'Freight_In' to float
freight_2018 = freight_2018.astype({'Month': np.int})
freight_2018 = freight_2018.astype({'Freight_In': np.float})

# Create output dataframe and sort by airline
freight = pd.DataFrame({'Airline': freight_2018.Airline.unique()})
freight = freight.sort_values('Airline').reset_index(drop=True)

# Calculate the 3 month rolling average and add to the output dataframe
for month in range(1,13):
    freight['Month'+str(month)] = 0.0
    for airline in freight.Airline:
        if month-2 > 0:
            curr = freight_2018[(freight_2018['Airline'] == airline) &
                                (freight_2018['Month'].between(month-2,month))]
            # Drop NaN values
            curr = curr.dropna()
            # Add rolling average (rounded) to output dataframe
            freight.loc[freight['Airline'] == airline, 
                        ['Month'+str(month)]] = round(curr['Freight_In'].mean(), 4)

# Inspect the results
display(freight.head())
display(freight.tail())

,Airline,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12
0,Air Caledonie International,0.0,0.0,3.8907,5.5907,4.8527,5.0463,3.2593,4.8170,4.7317,5.4093,4.1140,3.2510
1,Air Canada,0.0,0.0,282.2877,322.5487,386.1337,448.3783,442.0177,443.8630,382.5057,372.2470,339.4977,351.5970
2,Air China,0.0,0.0,718.6930,606.9110,573.7663,536.8107,473.7437,406.9380,364.5803,368.8330,367.9470,443.0510
3,Air India,0.0,0.0,161.2267,152.7690,131.2867,116.5900,121.3510,130.1677,153.7567,189.6320,224.0400,216.5800
4,Air Mauritius,0.0,0.0,46.8993,37.4673,35.4073,28.5680,30.4813,36.5367,50.3867,55.9920,54.0343,52.2957


,Airline,Month1,Month2,Month3,Month4,Month5,Month6,Month7,Month8,Month9,Month10,Month11,Month12
61,United Parcel Service,0.0,0.0,370.5505,363.1455,426.7367,433.0212,458.9222,439.1318,429.3661,454.7491,425.6438,399.3972
62,Vietnam Airlines,0.0,0.0,809.3293,876.9123,894.4057,902.7003,880.0663,896.2533,936.7647,942.6477,930.6030,859.6213
63,Virgin Australia,0.0,0.0,80.7243,86.4954,96.1866,104.9462,123.2874,149.8866,156.0185,153.4952,142.7361,140.2805
64,Virgin Samoa,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4200
65,Xiamen Airlines,0.0,0.0,352.4750,381.6967,458.9807,461.4870,485.7450,485.9420,493.6260,457.8633,468.7397,460.4883


The results look correct. Export the reuslts to a csv in the working directory.

In [15]:
freight.to_csv(output_path + "\\freight_avg_into_australia_2018.csv")